In [4]:
from docplex.mp.model import Model
import pandas as pd
# sava datas in a matrix of matrixes
first_data = [[0, 67, 23, 89], 
              [56, 0, 45, 23], 
              [12, 34, 0, 78], 
              [78, 90, 12, 0]]

second_data = [[10, 5, 5, 5], 
               [5, 5, 10, 5],
               [5, 5, 5, 10],
               [6, 10, 8, 10]]

third_data = [[0, 78, 12, 34], 
              [23, 0, 67, 45], 
              [89, 23, 0, 56], 
              [45, 78, 90, 0]]

data = [first_data, second_data, third_data]

# parameters
c = 4  # number of nodes
d = [8, 9, 10, 11]  # time intervals
tw = [[8, 12], [8, 12], [8, 12]]  # time windows
v = 2  # number of vehicles
M = 100000  # big M

# define model
mdl = Model("CVRP")

# decision variables
x = mdl.binary_var_cube(c, c, len(d) - 1, name="x")
T = mdl.continuous_var_list(c + 1, name="t")

# nodes constraints
mdl.add_constraint(
    mdl.sum(mdl.sum(x[0, j, t] for j in range(1, c)) for t in range(len(d) - 1)) == 1
)

mdl.add_constraint(
    mdl.sum(mdl.sum(x[j, 0, t] for j in range(1, c)) for t in range(len(d) - 1)) == 1
)

for j in range(1, c):
    mdl.add_constraint(
        mdl.sum(
            mdl.sum(x[i, j, t] for i in range(c) if i != j) for t in range(len(d) - 1)
        )
        == 1
    )


for j in range(1, c):
    mdl.add_constraint(
        mdl.sum(mdl.sum(x[i, j, t] for i in range(c)) for t in range(len(d) - 1) if i !=j) == mdl.sum(mdl.sum(x[j, i, t] for i in range(c)) for t in range(len(d) - 1) if i !=j)
    )

for t in range(len(d) - 1):
    mdl.add_constraint(
        T[0] >= d[t]*60*mdl.sum(x[0,j,t] for j in range(1,c))
    )

for i in range(c):
        for j in range(1,c):
            for t in range(len(d) - 1):
                if i != j:
                    mdl.add_constraint((1 - x[i, j, t]) * M +T[j]-T[i] >= data[t][i][j])

for t in range(len(d)-1):
    for i in range(1,c):
        mdl.add_constraint((1 - x[i, 0, t]) * M +T[c]-T[i] >=data[t][i][j])

for i in range(1, c):
    mdl.add_constraint(tw[i - 1][0] * 60 <= T[i])

for i in range(1, c):
    mdl.add_constraint(T[i] <= tw[i - 1][1] * 60)

# objective function
mdl.minimize(
    mdl.sum(
        mdl.sum(mdl.sum(x[i, j, t] * data[t][i][j] for i in range(c)) for j in range(c))
        for t in range(len(d) - 1)
    )
)

sol = mdl.solve()

if sol is None:
    print("No solution found")

print(sol.objective_value)
print(sol.get_values(T))

# print results
for t in range(len(d) - 1):
    print(f"\nTempo t = {t}")
    for i in range(c):
        for j in range(c):
            if sol.get_value(x[i, j, t]) == 1:
                print(f"  Percorso: da {i} a {j}")

21.0
[540.0, 550.0, 545.0, 555.0, 565.0]

Tempo t = 0

Tempo t = 1
  Percorso: da 0 a 2
  Percorso: da 1 a 3
  Percorso: da 2 a 1
  Percorso: da 3 a 0

Tempo t = 2
